# Laborator 6 – Regresie liniară cu metoda celor mai mici pătrate

## 🔍 Obiectiv
Scopul acestui laborator este să construim un sistem simplu de învățare automată capabil să prezică scorul fericirii unei populații în funcție de unii factori sociali și economici. Vom implementa algoritmul de regresie liniară de la zero, fără a folosi biblioteci specializate precum NumPy sau scikit-learn.

---

## 📊 Datele
Vom folosi datele din fișierul `v1_world-happiness-report-2017.csv`, care conține următoarele coloane relevante:
- **Happiness Score**: scorul total al fericirii
- **Economy (GDP per Capita)**: PIB-ul pe cap de locuitor
- **Family**: sprijinul social
- **Freedom**: libertatea de a lua decizii

---

## 🎯 Task-uri
1. Predicția scorului fericirii doar pe baza PIB-ului (demo)
2. Tema: Predicția scorului fericirii doar pe baza valorii "Family"
3. Tema: Predicția scorului fericirii pe baza PIB-ului și libertății
4. Implementare pe fișierele v2 și v3
5. (Opțional) Evaluarea pe date corupte sau incomplete

---

## ⚠️ Constrângeri
- Nu folosim biblioteci precum `numpy`, `sklearn`, `pandas` etc.
- Scriem funcții proprii pentru toate calculele.
- Se evaluează corectitudinea, claritatea și performanța algoritmului.

In [4]:
import csv

def load_data(file_path, x_columns):
    """
    Încarcă datele dintr-un fișier CSV și extrage coloanele relevante pentru X și coloana pentru Y.

    Args:
        file_path (str): calea către fișierul CSV
        x_columns (list[str]): o listă cu numele coloanelor independente (X)

    Returns:
        tuple: (X, Y), unde:
            - X este o listă de liste (fiecare sublistă conține valorile x pentru un exemplu)
            - Y este o listă cu valorile de predicție (scorul fericirii)
    """
    X = []
    Y = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                x_values = [float(row[col]) for col in x_columns]
                y_value = float(row["Happiness.Score"])
                X.append(x_values)
                Y.append(y_value)
            except ValueError:
                # Dacă o valoare lipseste sau nu e conversie validă, trecem peste rând
                continue
    return X, Y

In [7]:
import csv

with open("data/v1_world-happiness-report-2017.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    print("Header:", header)

# Doar pentru PIB
X_gdp, Y_gdp = load_data("data/v1_world-happiness-report-2017.csv", ["Economy (GDP per Capita)"])

# Doar pentru Family
X_family, Y_family = load_data("data/v1_world-happiness-report-2017.csv", ["Family"])

# Pentru PIB + Libertate
X_combo, Y_combo = load_data("data/v1_world-happiness-report-2017.csv", ["Economy (GDP per Capita)", "Freedom"])

Header: ['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Family', 'Health..Life.Expectancy.', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual']


KeyError: 'Economy (GDP per Capita)'

In [ ]:
def transpose(matrix):
    return [list(row) for row in zip(*matrix)]

def dot_product(vec1, vec2):
    return sum(a * b for a, b in zip(vec1, vec2))

def matrix_multiply(A, B):
    B_T = transpose(B)
    return [[dot_product(row_a, col_b) for col_b in B_T] for row_a in A]

def inverse_2x2(matrix):
    """ Inversă pentru o matrice 2x2 – doar pentru cazuri simple cu 1 sau 2 features """
    a, b = matrix[0]
    c, d = matrix[1]
    det = a * d - b * c
    if det == 0:
        raise ValueError("Matricea nu este inversabilă.")
    return [[d / det, -b / det],
            [-c / det, a / det]]

def least_squares_fit(X, Y):
    """
    Aplică metoda celor mai mici pătrate pentru regresie liniară.

    Args:
        X: lista de liste (m x n)
        Y: lista de m valori reale

    Returns:
        w: lista coeficienților [w0, w1, ..., wn]
    """
    # Adăugăm bias (coloană de 1)
    X_bias = [[1] + row for row in X]  # X devine m x (n+1)

    # X^T * X
    XT = transpose(X_bias)
    XTX = matrix_multiply(XT, X_bias)

    # X^T * Y (ca matrice coloană)
    XTY = [ [dot_product(row, Y)] for row in XT ]  # dimensiune (n+1) x 1

    # Inversăm XTX (merge doar pentru 2x2 aici; dacă sunt mai mulți factori, trebuie generalizat)
    if len(XTX) == 2:
        XTX_inv = inverse_2x2(XTX)
    else:
        raise NotImplementedError("Momentan doar 1 sau 2 features sunt suportate.")

    # Calculăm coeficienții w = (X^T * X)^(-1) * X^T * Y
    W = matrix_multiply(XTX_inv, XTY)  # (n+1) x 1

    return [w[0] for w in W]  # extragem coeficienții într-o listă simplă

In [ ]:
# Antrenăm modelul pe baza PIB-ului
weights_gdp = least_squares_fit(X_gdp, Y_gdp)
print("Coeficienți regresie (PIB):", weights_gdp)

# Antrenăm modelul pe baza Family
weights_family = least_squares_fit(X_family, Y_family)
print("Coeficienți regresie (Family):", weights_family)

# Antrenăm modelul pe baza PIB + Libertate
weights_combo = least_squares_fit(X_combo, Y_combo)
print("Coeficienți regresie (PIB + Libertate):", weights_combo)